<a href="https://colab.research.google.com/github/stratoskar/Path_Based_Traffic_Flow_Prediction/blob/main/Python_Code/1_Data_Collection_and_Route_Splitting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Data Collection**

You can read more about the dataset [here](https://stamen.com/work/cabspotting/)





In [ ]:
# Library to load data files
import os

# Data analysis libraries
import numpy as np
import pandas as pd

# Handle timestamps
from datetime import datetime, timedelta

In [ ]:
# Access Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Define the path to the folder with the data
PATH = '/content/drive/MyDrive/Paper/Datasets/Traffic Data'

Mounted at /content/drive


This dataset is a zip file that contains many separate files. Each one of the files contains the trajectory of one Yellow Taxi. For each taxi, the trajectory data are withing 17-05-2008 and 10-06-2008. All the taxis are moving inside the San Francisco city.

*To use the dataset, first unzip the file and run the code below.*

In [ ]:
# When iterating though the folder with the trajectory files, we read one file per time and add it in a single daraframe
# In order to keep track of the current trajectory, we assing a unique 'taxi_id' in the current trajectory data.
# Counter represents the current trajectory, so counter equals to 'taxi_id'.
counter = 0

# Create an empty dataframe, in which all the data will be saved
all_data = pd.DataFrame(columns=['taxi_id','Latitude','Longitude','Occupied','Date Time'])

for filename in os.listdir(PATH):

    try:
      # Read each file in the Data folder
      temp = pd.read_csv(PATH+'/'+filename,names=['Latitude','Longitude','Occupied','Date Time'],sep=' ')

      # Assign 'taxi_id' number to each file
      temp.insert(1,'taxi_id',counter)

      # Add the data in this file in the 'all_data' dataframe
      all_data = pd.concat([all_data, temp],ignore_index = True)

      # Increase 'taxi_id' number by 1
      counter += 1

      # Print file number ('taxi_id')
      print(f'File number is: {counter}')

    except: # If you meet any problem, move on
      continue

File number is: 1
File number is: 2
File number is: 3
File number is: 4
File number is: 5
File number is: 6
File number is: 7
File number is: 8
File number is: 9
File number is: 10
File number is: 11
File number is: 12
File number is: 13
File number is: 14
File number is: 15
File number is: 16
File number is: 17
File number is: 18
File number is: 19
File number is: 20
File number is: 21
File number is: 22
File number is: 23
File number is: 24
File number is: 25
File number is: 26
File number is: 27
File number is: 28
File number is: 29
File number is: 30
File number is: 31
File number is: 32
File number is: 33
File number is: 34
File number is: 35
File number is: 36
File number is: 37
File number is: 38
File number is: 39
File number is: 40
File number is: 41
File number is: 42
File number is: 43
File number is: 44
File number is: 45
File number is: 46
File number is: 47
File number is: 48
File number is: 49
File number is: 50
File number is: 51
File number is: 52
File number is: 53
Fi

In [ ]:
# Cast 'Date Time' column to timestamp
all_data['Date Time'] = pd.to_datetime(all_data['Date Time'],origin='unix',unit='s')

In [ ]:
# Sort all the data based in 'taxi_id' and 'Date Time' columns
all_data = all_data.sort_values(['taxi_id','Date Time'])

# Reset index, start indexing from zero
all_data = all_data.reset_index(drop=True)

**Delete the 'Occupied' column**

This column denotes whether or not the taxi was occupied by a passenger during GPS recording. So, this information is not useful for our research.

In [ ]:
# Drop column
all_data.drop('Occupied',axis=1,inplace=True)

**Present information about the dataset**

In [ ]:
# Show the shape of whole dataset
all_data.shape

(11220491, 4)

In [ ]:
# Print data types of every column present in the dataframe
all_data.dtypes

taxi_id              object
Latitude             object
Longitude            object
Date Time    datetime64[ns]
dtype: object

In [ ]:
# Show schema information of the dataframe
all_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5932540 entries, 0 to 11209686
Data columns (total 4 columns):
 #   Column     Dtype         
---  ------     -----         
 0   taxi_id    object        
 1   Latitude   object        
 2   Longitude  object        
 3   Date Time  datetime64[ns]
dtypes: datetime64[ns](1), object(3)
memory usage: 226.3+ MB


**Split the Routes of Taxis Into Trajectories**


In [ ]:
# Insert a new column
all_data.insert(1,'traj_id',-1)

In [ ]:
'''
Each taxi_id contains GPS data of one trajectory

If the time gap between two consequtive GPS points is lower than 'n_sec' seconds,
and these GPS points belong to the same 'taxi_id'
then asign the same 'traj_id' number.

If the time gap between two consequtive GPS points is higher than 'n_sec' seconds,
and these GPS points belong to the same 'taxi_id'
then asign different 'traj_id' number to each of these GPS points.

If the GPS points belong to the same 'taxi_id'
then asign different 'traj_id' number to each of these GPS points.

'''

# Max number of seconds between GPS records of each traectory
n_sec = 90
traj_id = 0

for i in range(all_data.shape[0] -1):

    # Belong to the same 'taxi_id'
    if (all_data['taxi_id'].iloc[i+1] == all_data['taxi_id'].iloc[i]):

        # Tme interval less-equal than 'n_sec'
        if (((all_data['Date Time'].iloc[i+1])-(all_data['Date Time'].iloc[i])).total_seconds() <= n_sec):
            all_data.at[i,'traj_id'] = traj_id
            all_data.at[i+1,'traj_id'] = traj_id

        # Time interval higher than 'n_sec'
        else:
            all_data.at[i,'traj_id'] = traj_id
            traj_id +=1
            all_data.at[i+1,'traj_id'] = traj_id

    # Not belong to the same 'taxi_id'
    else:
        all_data.at[i,'traj_id'] = traj_id
        traj_id  = 0
        all_data.at[i+1,'traj_id'] = traj_id

In [ ]:
# Save results to a csv file
SAVE_PATH = '/content/drive/MyDrive/Paper/splitted_data.csv'
all_data.to_csv(SAVE_PATH, index=False)